In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
import tensorflow as tf
import keras
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Embedding, Flatten, multiply,concatenate,Lambda
from tensorflow.keras.layers import Dense, Concatenate, Activation, Dropout,Reshape,Dot
from tensorflow.keras.regularizers import l2,l1
from tensorflow.keras.optimizers import SGD, Adamax,RMSprop,Adam
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint

In [412]:
data=pd.read_csv('br_data.csv');data

,User-ID,Location,Age,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher
0,183,portugal,27.0,8401422825,9,Fahrenheit 451,Ray Bradbury,1993,"Plaza &amp Janes Editores, S.A."
1,183,portugal,27.0,8440630794,7,Cuentos del Planeta Tierra,Arthur C. Clarke,1993,Ediciones B
2,183,portugal,27.0,8476409419,8,Estudios sobre el amor,Jose Ortega Y Gaset,2001,Downtown Book Center
3,242,germany,37.0,3257203659,9,Der illustrierte Mann. Erz?짚hlungen.,Ray Bradbury,2002,Diogenes
4,242,germany,37.0,3257207522,10,Der K?쨋nig in Gelb.,Raymond Chandler,1980,Diogenes Verlag
...,...,...,...,...,...,...,...,...,...
35548,278843,usa,28.0,1561580880,7,Easy Guide to Sewing Skirts (Sewing Companion ...,Marcy Tilton,1995,Taunton Press
35549,278851,usa,33.0,067161746X,7,The Bachelor Home Companion: A Practical Guide...,P.J. O'Rourke,1987,Pocket Books
35550,278851,usa,33.0,1558531025,8,Life's Little Instruction Book (Life's Little ...,H. Jackson Brown,1991,Thomas Nelson
35551,278851,usa,33.0,1569661057,10,"Dallas Street Map Guide and Directory, 2000 Ed...",Mapsco,1999,American Map Corporation


In [10]:
data=pd.read_csv('combined_data.csv');data

,User-ID,ISBN,Book-Rating,Book-Title,totalRatingCount,Location,Age
0,276762,3453092007,8,Die zweite Haut.,2,germany,25.0
1,35433,3453092007,8,Die zweite Haut.,2,switzerland,40.0
2,222488,3499121581,9,Rubinroter Dschungel.,6,germany,29.0
3,227176,3499121581,7,Rubinroter Dschungel.,6,germany,27.0
4,167166,3499121581,7,Rubinroter Dschungel.,6,germany,44.0
...,...,...,...,...,...,...,...
18876,179423,1570429227,7,Roses Are Red,2,usa,32.0
18877,179423,1570429693,10,Wish You Well,2,usa,32.0
18878,195914,3453180224,7,Lucy Sullivan wird heiraten.,3,germany,23.0
18879,195927,3462031791,9,"Aimee und Jaguar. Eine Liebesgeschichte, Berli...",3,germany,45.0


In [5]:
data.loc[:,'Age_c']=data['Age']//10

In [6]:
data['Age_c']=data['Age_c'].astype('int')

In [7]:
data['Age_c'].max()

9

In [8]:
data['User-ID'].nunique()

4198

In [231]:
# data=data.drop_duplicates(['ISBN']).reset_index(drop=True)

In [9]:
sc=LabelEncoder()
data['User-ID']=sc.fit_transform(data['User-ID'])

In [17]:
sc=LabelEncoder()
data['Location']=sc.fit_transform(data['Location'])

In [235]:
sc=LabelEncoder()
data['Book-Author']=sc.fit_transform(data['Book-Author'])

In [236]:
sc=LabelEncoder()
data['Publisher']=sc.fit_transform(data['Publisher'])

In [10]:
sc=LabelEncoder()
data['ISBN']=sc.fit_transform(data['ISBN'])

In [12]:
data['Age']=data['Age'].astype('int')

In [13]:
data

,User-ID,ISBN,Book-Rating,Book-Title,totalRatingCount,Location,Age,Age_c
0,4165,4565,8,Die zweite Haut.,2,germany,25,2
1,498,4565,8,Die zweite Haut.,2,switzerland,40,4
2,3324,4739,9,Rubinroter Dschungel.,6,germany,29,2
3,3398,4739,7,Rubinroter Dschungel.,6,germany,27,2
4,2496,4739,7,Rubinroter Dschungel.,6,germany,44,4
...,...,...,...,...,...,...,...,...
18876,2699,2611,7,Roses Are Red,2,usa,32,3
18877,2699,2613,10,Wish You Well,2,usa,32,3
18878,2942,4626,7,Lucy Sullivan wird heiraten.,3,germany,23,2
18879,2943,4678,9,"Aimee und Jaguar. Eine Liebesgeschichte, Berli...",3,germany,45,4


In [14]:
from sklearn.model_selection import train_test_split

X = data.drop('Book-Rating',axis=1)

Y = data['Book-Rating']

x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size = 0.3, random_state = 1234)

In [15]:
Y.mean()

7.849107568455061

In [18]:
# Variable 초기화 
K = 30                           # Latent factor 수 
reg = 0.05                      # Regularization penalty
mu = y_train.mean()    # 전체 평균 
M = data['User-ID'].max() + 1       # Number of users
N = data['ISBN'].max() + 1 
C = data['totalRatingCount'].max() + 1
A = data['Age_c'].max() + 1
L = data['Location'].max() + 1  # Number of movies
# A = data['Book-Author'].max() + 1  # Number of movies
# P = data['Publisher'].max() + 1

In [674]:
min_rating = min(Y)
max_rating = max(Y)

print(min_rating,max_rating)

1 10


In [448]:
A

10

In [396]:
# Defining RMSE measure
def RMSE(y_true, y_pred):
    return tf.sqrt(tf.reduce_mean(tf.square(y_true - y_pred)))

In [679]:
user = Input(shape=(1, )) # User input
item = Input(shape=(1, ))                                               # Item input
count = Input(shape=(1, )) 
age = Input(shape=(1, )) 
loc = Input(shape=(1, )) 



P_embedding = Embedding(M, K, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(user)     # (M, 1, K)
Q_embedding = Embedding(N, K, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(item)     # (N, 1, K)
C_embedding = Embedding(C, K, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(count)
A_embedding = Embedding(A, K, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(age)
L_embedding = Embedding(L, K, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(loc)
# P_embedding = Flatten()(P_embedding)                                    # (N, K)
# Q_embedding = Flatten()(Q_embedding)  

user_bias = Embedding(M, 1, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(user)       # User bias term (M, 1, )
item_bias = Embedding(N, 1, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(item)       # Item bias term (N, 1, )
count_bias = Embedding(C, 1, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(count)
age_bias = Embedding(A, 1, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(age)
loc_bias = Embedding(L, 1, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(loc)
# user_bias = Flatten()(user_bias)                                        # (N, K)
# item_bias = Flatten()(item_bias)   

R= multiply([P_embedding,Q_embedding,C_embedding,A_embedding,L_embedding])
R= concatenate([R,user_bias,item_bias,count_bias,age_bias,loc_bias])
# R = Activation('relu')(R)
R = Lambda(lambda x: x * (max_rating - min_rating) + min_rating)(R)
# R = Flatten()(R)
# R = Dense(10, activation="relu")(R)
# R= Dropout(0.3)(R)
R = Dense(1)(R)

model = Model(inputs=[user, item,count,age,loc], outputs=R)
model.compile(
  loss='mean_squared_error',
#    optimizer=SGD(lr=0.001, momentum=0.9),
#     optimizer=RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0),
#   optimizer=Adam(lr=0.001),
   optimizer='adam',
  metrics=[RMSE]
)

In [720]:
user = Input(shape=(1, )) # User input
item = Input(shape=(1, ))                                               # Item input
count = Input(shape=(1, )) 
age = Input(shape=(1, )) 
loc = Input(shape=(1, )) 



P_embedding = Embedding(M, K, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(user)     # (M, 1, K)
Q_embedding = Embedding(N, K, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(item)     # (N, 1, K)
# C_embedding = Embedding(C, K, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(count)
# A_embedding = Embedding(A, K, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(age)
# L_embedding = Embedding(L, K, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(loc)
# P_embedding = Flatten()(P_embedding)                                    # (N, K)
# Q_embedding = Flatten()(Q_embedding)  

user_bias = Embedding(M, 1, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(user)       # User bias term (M, 1, )
item_bias = Embedding(N, 1, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(item)       # Item bias term (N, 1, )
# count_bias = Embedding(C, 1, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(count)
# age_bias = Embedding(A, 1, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(age)
# loc_bias = Embedding(L, 1, embeddings_regularizer=l2(reg),embeddings_initializer='he_normal')(loc)
# user_bias = Flatten()(user_bias)                                        # (N, K)
# item_bias = Flatten()(item_bias)   

R= multiply([P_embedding,Q_embedding])
R= concatenate([R,user_bias,item_bias])
R = Lambda(lambda x: x * (max_rating - min_rating) - min_rating)(R)
# R = Flatten()(R)
# R = Dense(10, activation="relu")(R)
# R= Dropout(0.3)(R)
R = Dense(1)(R)

model = Model(inputs=[user, item], outputs=R)
model.compile(
  loss='mean_squared_error',
#    optimizer=SGD(lr=0.001, momentum=0.9),
#     optimizer=RMSprop(lr=0.01, rho=0.9, epsilon=None, decay=0.0),
#   optimizer=Adam(lr=0.001),
   optimizer='adam',
  metrics=[RMSE]
)

In [693]:
model.summary()

Model: "functional_303"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_439 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
input_440 (InputLayer)          [(None, 1)]          0                                            
__________________________________________________________________________________________________
embedding_864 (Embedding)       (None, 1, 30)        125940      input_439[0][0]                  
__________________________________________________________________________________________________
embedding_865 (Embedding)       (None, 1, 30)        166500      input_440[0][0]                  
_____________________________________________________________________________________

In [55]:
es = EarlyStopping(monitor='val_RMSE', mode='min', verbose=1, patience=5)
mc = ModelCheckpoint('best_model_multiply.h5', monitor='val_RMSE', mode='min', save_best_only=True)

In [721]:
history = model.fit(x=[x_train['User-ID'].values, x_train['ISBN'].values],y=y_train, batch_size= 128, epochs=60, 
                    verbose= 1, validation_data=([x_test['User-ID'].values, x_test['ISBN'].values], y_test))

Epoch 1/60
104/104 [==============================] - 0s 3ms/step - loss: 47.3356 - RMSE: 6.7635 - val_loss: 28.2674 - val_RMSE: 5.3067
Epoch 2/60
104/104 [==============================] - 0s 2ms/step - loss: 17.7218 - RMSE: 4.1416 - val_loss: 10.5024 - val_RMSE: 3.2115
Epoch 3/60
104/104 [==============================] - 0s 2ms/step - loss: 7.0477 - RMSE: 2.5988 - val_loss: 4.8760 - val_RMSE: 2.1626
Epoch 4/60
104/104 [==============================] - 0s 2ms/step - loss: 4.0782 - RMSE: 1.9713 - val_loss: 3.5061 - val_RMSE: 1.8269
Epoch 5/60
104/104 [==============================] - 0s 2ms/step - loss: 3.4428 - RMSE: 1.8114 - val_loss: 3.2208 - val_RMSE: 1.7567
Epoch 6/60
104/104 [==============================] - 0s 2ms/step - loss: 3.3145 - RMSE: 1.7822 - val_loss: 3.1483 - val_RMSE: 1.7423
Epoch 7/60
104/104 [==============================] - 0s 2ms/step - loss: 3.2776 - RMSE: 1.7789 - val_loss: 3.1223 - val_RMSE: 1.7386
Epoch 8/60
104/104 [==============================] - 0s 2

In [678]:
history = model.fit(x=[x_train['User-ID'].values, x_train['ISBN'].values,x_train['totalRatingCount'].values,x_train['Age_c'].values,x_train['Location'].values],y=y_train-mu, batch_size= batch_size, epochs=100, 
                    verbose= 1, validation_data=([x_test['User-ID'].values, x_test['ISBN'].values,x_test['totalRatingCount'].values,x_test['Age_c'].values,x_test['Location'].values], y_test-mu))

Epoch 1/100


C:\Users\gommg\AppData\Roaming\Python\Python38\site-packages\tensorflow\python\framework\indexed_slices.py:432: UserWarning: Converting sparse IndexedSlices to a dense Tensor of unknown shape. This may consume a large amount of memory.
  warnings.warn(


52/52 [==============================] - 0s 6ms/step - loss: 12.6553 - RMSE: 1.8747 - val_loss: 9.5091 - val_RMSE: 1.7858
Epoch 2/100
52/52 [==============================] - 0s 2ms/step - loss: 8.6612 - RMSE: 1.8397 - val_loss: 7.4814 - val_RMSE: 1.7681
Epoch 3/100
52/52 [==============================] - 0s 2ms/step - loss: 7.0383 - RMSE: 1.8250 - val_loss: 6.2229 - val_RMSE: 1.7603
Epoch 4/100
52/52 [==============================] - 0s 2ms/step - loss: 5.9970 - RMSE: 1.8149 - val_loss: 5.4010 - val_RMSE: 1.7573
Epoch 5/100
52/52 [==============================] - 0s 2ms/step - loss: 5.3006 - RMSE: 1.8109 - val_loss: 4.8350 - val_RMSE: 1.7537
Epoch 6/100
52/52 [==============================] - 0s 2ms/step - loss: 4.8139 - RMSE: 1.8056 - val_loss: 4.4252 - val_RMSE: 1.7477
Epoch 7/100
52/52 [==============================] - 0s 2ms/step - loss: 4.4605 - RMSE: 1.7992 - val_loss: 4.1238 - val_RMSE: 1.7417
Epoch 8/100
52/52 [==============================] - 0s 2ms/step - loss: 4.1934 

In [680]:
history = model.fit(x=[x_train['User-ID'].values, x_train['ISBN'].values,x_train['totalRatingCount'].values,x_train['Age_c'].values,x_train['Location'].values],y=y_train, batch_size= batch_size, epochs=100, 
                    verbose= 1, validation_split=.3)

Epoch 1/100
37/37 [==============================] - 0s 8ms/step - loss: 59.6609 - RMSE: 7.0078 - val_loss: 46.5119 - val_RMSE: 6.2754
Epoch 2/100
37/37 [==============================] - 0s 3ms/step - loss: 37.8430 - RMSE: 5.5994 - val_loss: 29.8614 - val_RMSE: 4.9520
Epoch 3/100
37/37 [==============================] - 0s 3ms/step - loss: 24.0295 - RMSE: 4.3822 - val_loss: 18.9955 - val_RMSE: 3.8490
Epoch 4/100
37/37 [==============================] - 0s 3ms/step - loss: 15.3096 - RMSE: 3.3821 - val_loss: 12.4322 - val_RMSE: 3.0137
Epoch 5/100
37/37 [==============================] - 0s 3ms/step - loss: 10.2929 - RMSE: 2.6956 - val_loss: 8.8553 - val_RMSE: 2.4749
Epoch 6/100
37/37 [==============================] - 0s 3ms/step - loss: 7.6350 - RMSE: 2.2655 - val_loss: 7.0044 - val_RMSE: 2.1790
Epoch 7/100
37/37 [==============================] - 0s 3ms/step - loss: 6.2639 - RMSE: 2.0442 - val_loss: 6.0191 - val_RMSE: 2.0325
Epoch 8/100
37/37 [==============================] - 0s 3ms/